# Multispectral Data Segmentation

![satellite](https://hackmd.io/_uploads/r1lA5vkx-l.jpg)

*Image generated by the DeepAI image generation tool.*

## Introduction

The Earth is constantly changing—cities are expanding, forest boundaries are shifting, and glaciers are melting. To understand these processes, scientists and engineers are increasingly turning to satellite data. Thanks to it, we can observe our planet from space, monitor the state of the environment, and detect changes that are often invisible to the "naked eye."

One of the key types of such data is multispectral imagery, which are photos taken in many different ranges of the electromagnetic spectrum. Each band provides different information about the Earth's surface, specifically about how a given material absorbs and reflects light. For example: in visible light, we see the natural colors of plants and soil, while infrared allows us to assess the condition of vegetation or soil moisture.

This technology enables the non-invasive creation of maps, analysis of agricultural land (e.g., for fertility), tracking the progress of droughts, as well as monitoring pollution or the effects of natural disasters. Today, multispectral images are one of the cornerstones of modern remote sensing.

Every material on Earth—water, sand, concrete, or grass—reflects radiation in its own characteristic way. We call this unique pattern a spectral signature. By analyzing these signatures in different bands, we can precisely identify the objects visible in the image.

In practice, however, this data can be distorted by atmospheric factors (clouds, water vapor, dust) and minor sensor measurement errors. Therefore, before multispectral images can be used for analysis, they must undergo appropriate processing and correction.

## Task

Your task is to prepare a class for processing the dataset and to train a model for segmenting multispectral images (terrain maps).

The first stage of the task involves creating a function that will process the input data in a way that maximizes the model's effectiveness (without using labels). To increase the diversity of the training set and improve generalization ability, you can use various data augmentation techniques, such as rotation, scaling, or adding noise (or other methods you deem appropriate).

During data analysis, try to select a **minimum** set of bands (channels) that will allow you to achieve high-quality segmentation. Consider which channels carry the most information—for example: does infrared affect the detection of vegetation? Using all bands is not always necessary; often, better results are achieved by selecting only those that contain key information for distinguishing between classes. Remember, you do not have to limit yourself to raw channel values—in the processing stage, you can generate entirely new feature representations.

The second stage is to create a solution for the segmentation of multispectral images, which means assigning one of four classes to each pixel: water, land, vegetation, or industrial areas (as in the example below). For this purpose, you can use libraries such as PyTorch and scikit-learn.

![segmentation_maps](https://live.staticflickr.com/65535/54927654414_b325b31a02_c.jpg)

You have at your disposal a labeled training and validation set on which you can test your approach. The final evaluation of the model will be carried out on a hidden test set. Each input image consists of 12 spectral bands, but the decision on how many and which of them you will use is up to you.

## Data Description

The data has been divided into separate sets:

- **training** - $48$ samples (images),

- **validation** - $16$ samples (images).

The final evaluation will use a **test** set consisting of $96$ samples (images), to which you do not have access.
Each sample is an image with dimensions of $30 \times 30$ pixels. Each pixel has an assigned class label (segmentation mask).
Each pixel is described by $12$ channels corresponding to measurements of light reflectance in different wavelength ranges.

## Evaluation Criteria

Your score depends on two factors: the quality of segmentation on the hidden test set and the number of channels you decide to use at the model's input.

The first component of the evaluation function is related to the number of channels used, $N\_channels$. Originally, each pixel is composed of 12 channels, and for using all channels, you will receive 0 points for this part of the task. All solutions using 3 or fewer channels will receive full points for this component of the evaluation function, while solutions using $\lbrace 4, 5, 6, ..., 11\rbrace$ channels will be evaluated according to a quadratic scaling function:

$$\mathtt{channel\_evaluation} = 
\begin{cases} 
    0 &\quad \text{if }  N\_channels \geq 12 \\
    100 &\quad \text{if }  N\_channels \leq 3 \\
    100 \cdot \left( \dfrac{12 - N\_channels}{12 - 3} \right)^2 &\quad \text{otherwise}.
\end{cases}$$

For example, for using 10 channels, you will receive only 5 points for this part of the task, multiplied by a factor of $0.25$.
The second component of the evaluation function is related to the quality of segmentation for the maps used in the task. We will use the $\text{IoU}$ (Intersection over Union) metric, which evaluates the ratio of correctly classified pixels of a given class in a given image to the total number of pixels of that class present in the ground truth map or the model's prediction. Finally, an average will be calculated over all classes and images in the test set. For each image, the average will be calculated only over the classes that appear in the *ground truth*.

$$\text{IoU} = \dfrac{1}{M \cdot N} \cdot \sum\limits_{i=1}^{N} \sum\limits_{j=1}^{M_i} \dfrac{\text{number of correctly classified pixels of the j-th class in the i-th map}}{\text{number of pixels of the j-th class in the combined ground truth and model prediction area for the i-th map}}$$

where $N$ is the number of images in a given set (e.g., test set), and $M_i$ is the number of classes actually present in the $i$-th map. If the average $\text{IoU}$ is no more than 0.6, you will receive 0 points for this part of the task, and if it is at least 0.8, you will receive the full score for this part.

$$\mathtt{segmentation\_evaluation} = 
\begin{cases} 
    0 &\quad \text{if }  \text{IoU} \leq 0.6 \\
    100 &\quad \text{if }  \text{IoU} \geq 0.8 \\
    100 \cdot \dfrac{\text{IoU} - 0.6}{0.8 - 0.6} &\quad \text{otherwise}
\end{cases}$$

An example of calculating $\text{IoU}$ for one of the classes (represented by blue squares) is shown in the image below. The final $\text{IoU}$ score is averaged over all classes present in a given image, as well as over all images in the set.

![IoU](https://live.staticflickr.com/65535/54922482577_1a5222581f_b.jpg)

The final score for the task will be $25\%$ from the evaluation for the number of channels used and $75\%$ from the points for segmentation quality, according to the formula:

$$\text{score} = 0.25 \cdot \mathtt{channel\_evaluation} + 0.75 \cdot \mathtt{segmentation\_evaluation}$$

**NOTE!** If the $\text{IoU}$ is less than 0.5, you will receive 0 points for the entire task, regardless of the number of channels used!


## Constraints

- Your solution will be tested on the Competition Platform without internet access and in an environment with a GPU.
- Model training and the evaluation of your final solution on the Competition Platform cannot take longer than 5 minutes using a GPU.
- The class performing the initial data transformation, `YourPreprocessing`, cannot use the dataset labels in any way (it can only process the samples themselves), and the segmentation must be performed directly in the `YourModel` class.

## Submission Files

This notebook completed with your solution (see the `YourModel` class and the `YourPreprocessing` class), in which you will prepare a setup that includes data transformation with potential channel reduction and modification, and a model that performs segmentation on the processed data.

## Evaluation

Remember that during the check, the `FINAL_EVALUATION_MODE` flag will be set to True.

You can score between 0 and 100 points for this task. The number of points you earn will be calculated on the (secret) test set on the Competition Platform based on the formula mentioned above, rounded to the nearest integer. If your solution does not meet the above criteria, does not execute correctly, or if an attempt at cheating is detected, you will receive 0 points for the task.

## Starter Code

In this section, we initialize the environment by importing the necessary libraries and functions. The prepared code will make it easier for you to operate on data effectively and build the right solution.

In [ ]:
######################### DO NOT CHANGE THIS CELL ##########################

FINAL_EVALUATION_MODE = False  # We will set this flag to True during evaluation.

In [ ]:
######################### DO NOT CHANGE THIS CELL ##########################

import os
import random
from tqdm import tqdm

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset, DataLoader

# Additional libraries you can use in your solution
import xgboost
import sklearn

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATA_DIR = "./data"
TRAIN_DATA_PATH = os.path.join(DATA_DIR, "train.npz")
VALID_DATA_PATH = os.path.join(DATA_DIR, "valid.npz")

assert torch.cuda.is_available(), "No graphics card (GPU) found!"

In [ ]:
######################### DO NOT CHANGE THIS CELL ##########################

seed = 12345

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
os.environ["PYTHONHASHSEED"] = str(seed)

### Data Loading
Using the code below, we load data containing multispectral images and their corresponding segmentation masks. This data will be the basis for training and validating your segmentation model.

In [ ]:
######################### DO NOT CHANGE THIS CELL ##########################
# A cell containing auxiliary functions for data preparation.

class BaseDataset(Dataset):
    """
    Multispectral dataset class.
    """
    def __init__(self, data_path: str):
        data = np.load(data_path)
        self.bands = data["bands"]
        self.segmentations = data["segmentations"]

    def __len__(self):
        """Returns the number of samples in the dataset."""
        return self.bands.shape[0]

    def __getitem__(self, idx):
        """Returns a sample at index idx - its multispectral bands and segmentation."""
        bands = self.bands[idx]
        segmentations = self.segmentations[idx]

        bands = torch.from_numpy(bands).float()
        segmentations = torch.from_numpy(segmentations).long()

        return bands, segmentations

def setup_data():
    """
    Downloads the datasets used in the task.
    """
    import gdown
    os.makedirs(DATA_DIR, exist_ok=True)

    if not os.path.exists(TRAIN_DATA_PATH):
        url = "https://drive.google.com/uc?id=1Nfv3Kd9W8ypjr8RL1jY3VF3yTKU934lz"
        gdown.download(url, TRAIN_DATA_PATH, fuzzy=True)
    
    if not os.path.exists(VALID_DATA_PATH):
        url = "https://drive.google.com/uc?id=1WUwayYErm4CXI7zHUAmZmi23doIKmDm3"
        gdown.download(url, VALID_DATA_PATH, fuzzy=True)

if not FINAL_EVALUATION_MODE:
    setup_data()

### Code with the Evaluation Criterion

Code similar to the one below will be used to evaluate the solution on the test set.

In [ ]:
######################### DO NOT CHANGE THIS CELL ##########################

def calculate_miou(y_true, y_pred):
    """
    Calculates the mIoU (mean Intersection over Union) metric used to evaluate the model.
    A helper function used in the evaluation of the solution.
    """
    assert y_true.shape == y_pred.shape
    assert y_true.device == y_pred.device

    num_classes = 4
    device = y_true.device
    batch_size = y_true.shape[0]
    y_true_flat = y_true.reshape(batch_size, y_true.shape[1] * y_true.shape[2])
    y_pred_flat = y_pred.reshape(batch_size, y_pred.shape[1] * y_pred.shape[2])

    # The final average is calculated only for classes present in the ground truth data
    intersections = torch.zeros((batch_size, num_classes), dtype=torch.float32, device=device)
    unions = torch.zeros((batch_size, num_classes), dtype=torch.float32, device=device)
    true_present = torch.zeros((batch_size, num_classes), dtype=torch.bool, device=device)

    for cls in range(num_classes):
        y_true_c = (y_true_flat == cls)
        y_pred_c = (y_pred_flat == cls)
        true_present[:, cls] = torch.any(y_true_c, dim=1)

        intersections[:, cls] = torch.sum(y_true_c & y_pred_c, dim=1).to(torch.float32)
        unions[:, cls] = torch.sum(y_true_c | y_pred_c, dim=1).to(torch.float32)

    # Calculates the number of relevant classes for each sample
    num_present_classes = torch.sum(true_present, dim=1).to(torch.float32)

    # Although calculations are performed for all classes, we only sum those that are actually present (in the ground truth data)
    iou_per_class = torch.nan_to_num(intersections / unions, nan=0.0)
    sum_iou = torch.sum(iou_per_class * true_present, dim=1)
    miou_scores = torch.nan_to_num(sum_iou / num_present_classes, nan=0.0).tolist()
    assert len(miou_scores) == batch_size

    return miou_scores


def calculate_channel_count(dataloader):
    """
    A helper function used in the evaluation of the solution.
    Calculates the number of bands used during model training and evaluation.
    """
    tensors = dataloader.dataset.tensors
    
    if len(tensors) != 2:
        raise ValueError("The dataset should only contain labels and images (__getitem__ should return a tuple of dimensionality 2)")
    
    x, _ = tensors
    
    # x.shape [dataset size, channels, height, width]
    if x.ndim != 4:
        raise ValueError("Processed data must have 4 dimensions [batch size, channels, height, width]")

    if x.shape[0] < 15: #the validation set is the smallest and has 15 samples
        raise ValueError(f"The first dimension is reserved for the dataset size, your code should not change it!")

    if x.shape[2] != 30 or x.shape[3] != 30:
        raise ValueError("Processed data must have dimensions 30x30 on the 3rd and 4th dimension (.shape[2] == .shape[3] == 30)")

    channels_count = x.shape[1]
    return channels_count

def transform_dataset(preprocessing, dataset:BaseDataset) -> TensorDataset:
    """
    Processes the specified dataset and stores it in the computer's memory (RAM).
    Calls the .transform function implemented by the participant in the class that prepares datasets.
    """
    processed_labels = []
    processed_images = []
    
    for raw_image, label in dataset:
        processed_labels.append(label.clone())
        
        transformed_image = preprocessing.transform(raw_image.clone())
        processed_images.append(transformed_image)

    labels_tensor = torch.stack(processed_labels) 
    images_tensor = torch.stack(processed_images)  
    memory_dataset = TensorDataset(images_tensor, labels_tensor)
    return memory_dataset

def evaluate(train_model, preprocessing, data_path: str):
    """
    Main function for evaluating the task.
    The same function will be called on the Competition Platform.

    1. Fits the data preprocessing class on the training set.
    2. Processes the specified dataset using the fitted class.
    3. Evaluates the model using the mIoU metric on the specified processed dataset.
    4. Evaluates the number of bands used for model evaluation - checks the data shape.
    """

    # Loads the training set and fits the data preparation class on it
    train_ds = BaseDataset(data_path=TRAIN_DATA_PATH)
    preprocessing = preprocessing()
    assert hasattr(preprocessing, "fit"), "The data preparation function must implement the .fit method"
    preprocessing.fit(train_ds)
    
    # Loads the target dataset and processes it using the data preparation class
    target_ds = BaseDataset(data_path=data_path)
    assert hasattr(preprocessing, "transform"), "The data preparation function must implement the .transform method"
    transformed_dataset = transform_dataset(preprocessing=preprocessing, dataset=target_ds)
    dataloader = DataLoader(transformed_dataset, batch_size=8, shuffle=False)

    model = train_model()

    if hasattr(model, "to") and callable(getattr(model, "to", None)):
        model = model.to(DEVICE)

    if hasattr(model, "eval") and callable(getattr(model, "eval", None)):
        model.eval()

    mious = []

    # Evaluates the model using the mIoU metric
    with torch.no_grad():
        
        for x, y in dataloader:
            x = x.to(DEVICE)
            y = y.to(DEVICE)

            y_pred = model(x)
            if hasattr(y_pred, "to") and callable(getattr(y_pred, "to", None)):
                y_pred = y_pred.to(DEVICE)
            if not torch.is_tensor(y_pred):
                y_pred = torch.tensor(y_pred)
            y_pred = torch.argmax(y_pred, dim=1)

            assert y_pred.shape == y.shape
            assert y_pred.max() <= 4 and y_pred.min() >= 0

            miou = calculate_miou(y, y_pred)
            mious.extend(miou)

    # Evaluates the number of bands used for model evaluation
    channels_count = calculate_channel_count(dataloader=dataloader)
    
    # Calculates the average mIoU over all samples
    miou = sum(mious) / len(mious)

    return miou, channels_count
    
def compute_score(miou, channels_count):
    """
    Calculates the score for the task, calculates the final number of points for the task using the formula given in the task description.
    The same function will be called on the Competition Platform.
    """
    band_score = max(0, min(100, 100 * ((12 - channels_count) / (12 - 3))**2))
    miou_score = max(0, min(100, 100 * (miou - 0.6) / (0.8 - 0.6)))

    print(f"Number of channels: {channels_count}")
    print(f"mIoU: {miou:.3f} \n")

    print(f"Score for channels: {band_score:.3f}")
    print(f"Score for mIoU: {miou_score:.3f}")

    if miou_score < 0.5:
        total_score = int(0)
    else:
        total_score = 0.25 * band_score + 0.75 * miou_score
        total_score = int(round(total_score))
    print(f"Estimated number of points for the task: {total_score}")
    return total_score

## Sample Solution

Below we present a simplified solution based on linear regression, which can serve as an example demonstrating the operation of the notebook as well as a starting point for creating your solution.

In [ ]:
######################### DO NOT CHANGE THIS CELL ##########################
class BasicPreprocessing():
  """
  Sample implementation of a dataset processing class.

  This is just an example solution that does not reduce the number of channels.
  The fewer channels you use, the more points you will receive for this part of the assignment.
  """

  def __init__(self):
    self.std_per_channel = None
    
  def fit(self, dataset:BaseDataset):

    # Zbiera wszystkie obrazy do jednej macierzy [N_próbek, 12, 30, 30]
    images = []
    for item in dataset:
        image_tensor, _ = item
        images.append(image_tensor)
    images_tensor = torch.stack(images)  # shape: [N, 12, 30, 30]

    # Oblicza std (odchylenie standardowe) po wymiarach: (0: próbka, 2: H, 3: W)
    self.std_per_channel = images_tensor.std(dim=(0, 2, 3))

    return self

  def transform(self, image_tensor: torch.Tensor) -> torch.Tensor:          
    # Changes the shape of the mean to [12, 1, 1] to allow broadcasting relative to [12, 30, 30].
    # Remember, this is just an example solution, simply dividing by std is not the most efficient solution.      std_reshaped = self.std_per_channel.view(-1, 1, 1)
     
    return image_tensor / std_reshaped

In [ ]:
######################### DO NOT CHANGE THIS CELL ##########################

class BasicModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(12, 4)
    
    def forward(self, bands):
        # bands.shape [b, c, h, w]
        return self.linear(bands.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)

### Sample Model Training

In [ ]:
######################### DO NOT CHANGE THIS CELL ##########################

def train_basic_model():

    epochs = 40
    lr = 0.001
    batch_size = 8
    model = BasicModel()
    model = model.to(DEVICE)

    # Prepares data preprocessing parameters only using the
    # training set.

    raw_train_ds = BaseDataset(data_path=TRAIN_DATA_PATH)
    raw_valid_ds = BaseDataset(data_path=VALID_DATA_PATH)

    preprocessing = BasicPreprocessing()
    preprocessing.fit(raw_train_ds)

    train_ds = transform_dataset(preprocessing=preprocessing, dataset=raw_train_ds)
    valid_ds = transform_dataset(preprocessing=preprocessing, dataset=raw_valid_ds)
    train_dataloader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    valid_dataloader = DataLoader(valid_ds, batch_size=batch_size, shuffle=False)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        for x, y in tqdm(train_dataloader, total=len(train_dataloader), desc="Training"):
            x = x.to(DEVICE)
            y = y.to(DEVICE)

            y_pred = model(x)
            loss = criterion(y_pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        model.eval()
        with torch.no_grad():
            valid_loss = 0
            mious = []
            for x, y in tqdm(valid_dataloader, total=len(valid_dataloader), desc="Validation"):
                x = x.to(DEVICE)
                y = y.to(DEVICE)

                y_pred = model(x)
                loss = criterion(y_pred, y)

                valid_loss += loss.item()

                y_pred = torch.argmax(y_pred, dim=1)
                miou = calculate_miou(y, y_pred)
                mious.extend(miou)
            
            valid_loss = valid_loss / len(valid_dataloader)
            print(f"Epoch {epoch+1} loss: {valid_loss}, mIoU: {sum(mious) / len(mious)}")
            
    return model

### Evaluation of the Sample Solution

In [ ]:
######################### DO NOT CHANGE THIS CELL ##########################

if not FINAL_EVALUATION_MODE:
    miou, channels_count = evaluate(train_basic_model, BasicPreprocessing, VALID_DATA_PATH)
    print("-"*50)
    compute_score(miou, channels_count)

## Your Solution
Your solution should be in this section. Please make changes here only!

In [ ]:
# Do not change the class name.
# This class can only process samples, it cannot use data labels.
# Neither method can perform segmentation.

class YourPreprocessing():
  def __init__(self):
    pass
    
  def fit(self, dataset: BaseDataset):
    return dataset

  def transform(self, image_tensor: torch.Tensor) -> torch.Tensor:
    return image_tensor

In [ ]:
class YourModel(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, bands):
        # random predictions
        segmentation = torch.randint(0, 4, (bands.shape[0], 1, 30, 30))
        # We perform one hot encoding because evaluation uses torch.argmax
        one_hot = torch.zeros(bands.shape[0], 4, 30, 30)
        one_hot.scatter_(1, segmentation, 1)
        return one_hot

In [ ]:
# Don't change the function name
def train_your_model():
    return YourModel()

## Evaluation

Running the cell below will allow you to check how many points your solution would score on the validation data. On the Competition Platform, your solution will be evaluated on the test set.

Before submitting, make sure that the entire notebook runs from start to finish without errors and without user intervention after executing the `Run All` command.

In [ ]:
######################### DO NOT CHANGE THIS CELL ##########################

if not FINAL_EVALUATION_MODE:
    miou, channels_count = evaluate(train_your_model, YourPreprocessing, VALID_DATA_PATH)
    print("-"*50)
    compute_score(miou, channels_count)

**Remember:** During the evaluation, the model (the model training function) and the data processing function will be evaluated on the test set, not the validation set!